### **Resumen Técnico del Proyecto**

Este proyecto se divide en dos componentes principales: el **backend de Machine Learning**, desarrollado en un notebook de Jupyter (`modelo.ipynb`), y el **frontend interactivo**, construido como una aplicación web con Streamlit (`app.py`).

---

### **Archivo 1: `modelo.ipynb` - Creación y Entrenamiento del Modelo de IA**

Este notebook es el corazón del Objetivo 3. Su propósito es transformar los datos crudos de ECG en un dataset estructurado y entrenar un modelo de clasificación.

#### **Paso 1: Ingeniería de Características y Creación del Dataset**

En lugar de usar la señal de ECG completa (lo que requeriría modelos más complejos como CNNs o RNNs), se adoptó un enfoque de **ingeniería de características**. Se extrajeron dos características numéricas clave de cada registro para simplificar el problema:

1.  **Frecuencia Cardiaca Promedio:**
    *   **Tecnología:** Se utilizó la librería `neurokit2`.
    *   **Proceso:** Para cada registro, se aísla la derivación II, se limpia la señal para reducir el ruido (`nk.ecg_clean`) y se detectan los picos R (`nk.ecg_peaks`). A partir de estos picos, se calcula la frecuencia cardiaca instantánea y luego el promedio.
    *   **Robustez:** El proceso es robusto, ya que filtra valores de FC absurdos (fuera del rango 30-200 lpm) y maneja registros donde no se pueden detectar picos.

2.  **Rango Cardiaco:**
    *   Es una característica **categórica binaria** derivada de la anterior. Se etiqueta como "Dentro" si la FC está entre 60-100 lpm, y "Fuera" en caso contrario. Esto le da al modelo un contexto clínico directo.

3.  **Etiqueta de Destino (Target):**
    *   **Proceso:** Se leen los códigos de diagnóstico SNOMED CT de cada archivo `.hea`. Se mapean los acrónimos (ej. 'SB', 'SR', 'AFIB', 'ST') a las cuatro clases de interés del proyecto.
    *   **Limpieza de Datos:** Los registros que no pertenecían a ninguna de estas cuatro clases se etiquetaron como "Otro" y **fueron eliminados del dataset de entrenamiento**. Esto es crucial para enfocar el modelo únicamente en las clases relevantes.

El resultado final de este paso es un archivo `ecg_classification_dataset.csv` con características limpias y listas para el entrenamiento.

#### **Paso 2: Entrenamiento del Modelo de Red Neuronal**

Se entrenó un modelo de clasificación supervisada con las características generadas.

*   **Modelo:** Se utilizó una **Red Neuronal Artificial (ANN)**, específicamente un Perceptrón Multicapa (MLP), implementado con `TensorFlow/Keras`.
*   **Arquitectura:** Es un modelo secuencial simple pero efectivo para datos tabulares:
    *   **Capas de Entrada y Ocultas:** Varias capas `Dense` (totalmente conectadas) con activación `ReLU`.
    *   **Regularización:** Se usó `Dropout` y `BatchNormalization` entre las capas para prevenir el sobreajuste (overfitting) y estabilizar el entrenamiento.
    *   **Capa de Salida:** Una capa `Dense` con 4 neuronas (una por cada clase) y activación `softmax`, que convierte las salidas en un vector de probabilidades que suman 1.
*   **Preprocesamiento para el Modelo:**
    *   **`LabelEncoder`:** Convirtió las etiquetas de texto (ej. "Sinus Bradycardia") en números enteros (0, 1, 2, 3) que el modelo puede entender.
    *   **`StandardScaler`:** Normalizó las características de entrada (FC y Rango) para que tuvieran media cero y desviación estándar de uno. Esto es fundamental para que las redes neuronales converjan eficientemente.
*   **Entrenamiento:**
    *   **Función de Pérdida:** `sparse_categorical_crossentropy`, ideal para clasificación multiclase con etiquetas enteras.
    *   **Optimizador:** `Adam`, un optimizador robusto y ampliamente utilizado.
    *   **Métrica:** `accuracy` (precisión).
*   **Artefactos Guardados:**
    1.  `ecg_classifier_heart_rate.h5`: El modelo entrenado, incluyendo su arquitectura y pesos.
    2.  `label_encoder_classes.npy`: Un archivo que guarda las clases del `LabelEncoder` en el orden correcto. Esto es **esencial** para que la aplicación web pueda decodificar correctamente las predicciones del modelo.

---

### **Archivo 2: `app.py` - La Aplicación Web Interactiva**

Esta aplicación integra todos los objetivos del proyecto en una interfaz de usuario amigable.

*   **Tecnología Principal:** **Streamlit**, un framework de Python para crear aplicaciones de datos.
*   **Optimización:** Usa decoradores como `@st.cache_data` y `@st.cache_resource` para evitar recargar datos y modelos en cada interacción, haciendo la app más rápida.

#### **Implementación de Objetivos:**

*   **Objetivo 1 (Visualización):**
    *   **Tecnología:** `Plotly`.
    *   **Detalle Técnico:** Se crean 12 subplots y se configura la cuadrícula (`dtick` y `minor.dtick`) para simular con precisión el papel de ECG estándar (0.2s/0.5mV y 0.04s/0.1mV). El uso de Plotly añade interactividad (zoom, pan).

*   **Objetivo 2 (Análisis de FC):**
    *   **Tecnología:** `neurokit2`.
    *   **Proceso:** El flujo es idéntico al del notebook, pero se aplica en tiempo real al registro seleccionado por el usuario.
    *   **Robustez:** La aplicación maneja correctamente los casos donde no se detectan picos R, mostrando "N/A" y previniendo errores, lo que la hace más fiable.

*   **Objetivo 3 (Clasificación con IA):**
    *   **Carga del Modelo:** Al iniciar, la app carga los archivos `ecg_classifier_heart_rate.h5` y `label_encoder_classes.npy`.
    *   **Proceso de Predicción (al hacer clic en el botón):**
        1.  Toma la **Frecuencia Cardiaca Promedio** y el **Rango Cardiaco** calculados en el Objetivo 2.
        2.  Prepara estos dos valores como un array de características.
        3.  **Aplica una normalización manual** para que coincida con la que se usó en el entrenamiento.
        4.  Llama a `model.predict()` con estos datos.
        5.  Usa el `label_encoder` cargado para traducir la salida numérica del modelo (ej. `1`) a la etiqueta de texto correspondiente (ej. "Sinus Bradycardia").
    *   **Visualización de Resultados:** Muestra la clase predicha, la confianza del modelo y dos gráficos (barras y pie) creados con `matplotlib` para visualizar las probabilidades de cada clase.

---

### **Posibles Preguntas y Respuestas Técnicas**

1.  **P: ¿Por qué usaron un modelo MLP simple con solo dos características en lugar de una CNN con la señal completa?**
    *   **R:** Fue una decisión de diseño para cumplir con el bonus de forma eficiente. El enfoque de **ingeniería de características** nos permitió crear un modelo más simple y rápido de entrenar, que aun así captura la información esencial (la frecuencia cardiaca) para diferenciar estas cuatro arritmias específicas, que están directamente relacionadas con el ritmo (lento, normal, rápido o irregular). Una CNN requeriría un preprocesamiento de la señal mucho más complejo y un dataset significativamente más grande para entrenar sin sobreajuste.

2.  **P: El reporte de clasificación muestra un rendimiento bajo para "Sinus Rhythm" y "Sinus Tachycardia". ¿A qué se debe y cómo lo mejorarían?**
    *   **R:** Se debe principalmente al **desbalance de clases** en nuestro dataset de entrenamiento, donde "Sinus Bradycardia" era la clase mayoritaria. Con pocas muestras de las otras clases, el modelo no pudo aprender a distinguirlas bien.
    *   **Mejoras:**
        1.  **Recolectar más datos** para balancear las clases.
        2.  **Usar técnicas de sobremuestreo** (como SMOTE) en las clases minoritarias.
        3.  **Añadir más características** al modelo, como la variabilidad de la frecuencia cardiaca (desviación estándar de los intervalos RR), que es un buen indicador de Fibrilación Auricular.

3.  **P: En `app.py`, veo que la normalización de los datos para la predicción está "hardcodeada" (escrita manualmente). ¿Es esta la mejor práctica?**
    *   **R:** Es una excelente observación. No es la mejor práctica. Lo hicimos así por simplicidad en la implementación. El enfoque ideal sería **guardar el objeto `StandardScaler`** del notebook (usando una librería como `pickle` o `joblib`) y cargarlo en la aplicación de Streamlit. De esa manera, se aplicaría exactamente la misma transformación que en el entrenamiento, haciendo el sistema más robusto y fácil de mantener si el modelo se reentrena.

4.  **P: ¿Por qué eligieron `neurokit2` para el análisis de picos R?**
    *   **R:** Elegimos `neurokit2` porque es una librería de código abierto, validada científicamente y especializada en el procesamiento de bioseñales. Sus algoritmos para la detección de picos R son robustos e incluyen pasos de pre-limpieza de la señal, lo que nos da resultados más fiables que si hubiéramos intentado implementar un detector de picos desde cero.